In [9]:
from dotenv import load_dotenv
from rich import print as rich_print

load_dotenv()

True

In [10]:
import ollama
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.ollama import OllamaProvider

OLLAMA_MODEL_NAME = "gpt-oss:20b"
OLLAMA_BASE_URL = "http://localhost:11434"

ollama.pull(OLLAMA_MODEL_NAME)

model = OpenAIChatModel(
    model_name=OLLAMA_MODEL_NAME,
    provider=OllamaProvider(base_url=f"{OLLAMA_BASE_URL}/v1", api_key="ollama"),
)

In [11]:
with open("GEORGE_WASHINGTON_SHORT.md", "r", encoding="utf-8") as f:
    bio = f.read()


In [12]:
from pydantic import BaseModel
from dataclasses import dataclass
from pydantic_ai import Agent, RunContext, NativeOutput


@dataclass
class Biography:
    text: str


class PersonalExperience(BaseModel):
    title: str
    start_year: int
    end_year: int
    description: str
    major_accomplishments: list[str]
    skills: list[str]

class StructuredBio(BaseModel):
    experiences: list[PersonalExperience]
    education: list[str]
    skills: list[str]
    accomplishments: list[str]

agent = Agent[None, StructuredBio](
    model,
    deps_type=Biography,
    output_type=NativeOutput(StructuredBio),
)

@agent.instructions
async def instructions(ctx: RunContext[Biography]) -> str:
    return f"""
    You are a helpful assistant that can analyze a biography and extract structured information about the person's personal experiences, education, skills, and accomplishments.

    Here is the biography:
    {ctx.deps.text}
    """



In [13]:
result = await agent.run(deps=Biography(text=bio))

structured_bio = result.output
rich_print(structured_bio)
rich_print(result.usage())


StructuredBio(
    experiences=[
        PersonalExperience(
            title='Early Life & Military Beginnings',
            start_year=1732,
            end_year=1775,
            description='Born to a Virginia planter family; became a surveyor for Lord Fairfax; appointed major, 
then lieutenant colonel in the provincial militia; led 1754 attack on Fort\u202fDuquesne; fought in French 
&\u202fIndian War; resigned in 1758 to manage plantation and serve in Virginia House of Burgesses.',
            major_accomplishments=[
                'First officer to deliver ultimatum to French at Fort\u202fDuquesne',
                'Commanded Virginia forces after General Braddock’s death'
            ],
            skills=['Frontier surveying, land measurement', 'Early military leadership', 'Tactical diplomacy']
        ),
        PersonalExperience(
            title='Planter & Family Life',
            start_year=1755,
            end_year=1775,
            description='Ran Mount\u202fVernon plantation; married Martha Dandridge Custis in 1759, creating one of
the most prominent families in Virginia; maintained a successful agricultural enterprise while developing his 
political network.',
            major_accomplishments=['Sustained a large plantation', 'Established strong social ties'],
            skills=['Agricultural management', 'Estate administration', 'Civic leadership']
        ),
        PersonalExperience(
            title='Revolutionary Leadership',
            start_year=1774,
            end_year=1783,
            description='Delegate to the First Continental Congress (1774); elected Commander‑in‑Chief of the 
Continental Army (1775–1783); led the army through the Revolutionary War, winning key victories and negotiating the
Treaty of Paris.',
            major_accomplishments=[
                'Defense of Boston, New York &\xa0Philadelphia',
                'Surprise victories at Trenton &\xa0Princeton',
                'Surrender of Cornwallis at Yorktown',
                'Voluntary resignation of military commission after victory'
            ],
            skills=[
                'Military strategy & logistics',
                'Crisis management',
                'Reconciliation & restraint',
                'Diplomatic negotiation',
                'Public persuasion'
            ]
        ),
        PersonalExperience(
            title='Founding Father/Constitutional Architect',
            start_year=1787,
            end_year=1789,
            description='Presided over the Constitutional Convention; leveraged his prestige to secure ratification
of the Constitution; elected as the first President of the United States.',
            major_accomplishments=[
                'Presiding officer of the Constitutional Convention',
                'Orchestrated ratification efforts',
                'First elected President'
            ],
            skills=['Executive decision‑making', 'Consensus building', 'Political persuasion']
        ),
        PersonalExperience(
            title='Presidency (First Term)',
            start_year=1789,
            end_year=1793,
            description='Set foundational precedents; signed major legislative acts establishing federal 
institutions; established the First Bank; suppressed the Whiskey Rebellion; led the navy’s early development.',
            major_accomplishments=[
                'Judiciary Act, Naturalization Act, Residence Act, Copyright Act, Bank Act',
                'Signed bill creating First Bank of the U.S.',
                'Signed act establishing U.S. Mint and official currency',
                'Authorized construction of six frigates'
            ],
            skills=[
                'Legislative advocacy',
                'Fiscal policy design',
                'National security leadership',
                'Public relations'
            ]
        ),
        PersonalExperience(
            title='Presidency (Second Term)',
            start_

RunUsage(input_tokens=2120, output_tokens=1093, requests=1)

In [16]:
from pydantic_yaml import to_yaml_str


str_agent = Agent(model)

result = await str_agent.run(f"""
Convert the following information about George Washington into a resume suitable for a job application.
output in markdown format.

{to_yaml_str(structured_bio)}
""")

resume = result.output
rich_print(resume)


# George Washington  
Executive Leader | Military Strategist | Nation‑Builder  
`Available for senior executive or advisory roles in government, defense, or institutional development.`  

## Professional Summary  
A 19th‑century pioneer in military command, constitutional architecture, and national institution building. Led the
Continental Army to decisive victories, presided over the Constitutional Convention, and served as the First 
President of the United States, establishing enduring federal protocols and institutions. Proven talent in 
large‑scale logistics, fiscal policy design, diplomatic negotiation, and visionary public persuasion. Seeking to 
leverage this legacy of strategic leadership and institutional stewardship in a modern senior‑executive or advisory
capacity.

---

## Professional Experience  

**Founder & President – United States of America**  
*First & Second President* (1789‑1797)  
- Set presidential precedents that remain the cornerstone of U.S. executive practice.  
- Established key federal institutions: Supreme Court, Congress, Navy, Mint, and the First Bank of the U.S.  
- Signed landmark legislation—Judiciary Act, Naturalization Act, Residence Act, Copyright Act, and Bank Act—laying 
the foundations of American law and finance.  
- Suppressed the Whiskey Rebellion, reinforcing federal authority and national unity.  
- Issued the Proclamation of Neutrality (1793), preserving U.S. independence during European conflict.  
- Negotiated the Jay Treaty (1794) and Treaty of San Lorenzo (1795), securing trade and frontier peace.  
- Published the Farewell Address, defining a national policy of foreign‑policy restraint.

**Leader, Continental Army – United States Revolutionary War**  
*Commander‑in‑Chief* (1775‑1783)  
- Orchestrated the strategic defence of Boston, New York, and Philadelphia.  
- Achieved surprise victories at Trenton & Princeton; secured the surrender of Cornwallis at Yorktown (1781).  
- Directed comprehensive logistics, intelligence, and morale programs across a multinational frontier.  
- Voluntarily relinquished military commission in 1783, demonstrating political restraint and commitment to 
civilian governance.

**Delegate – First Continental Congress & Constitutional Convention**  
*Presiding Officer* (1774‑1787)  
- Leveraged prestige to secure unanimous ratification of the U.S. Constitution.  
- Guided the drafting process that balanced state sovereignty with federal authority.  

**Surveyor & Early Military Officer – Virginia Provincial Militia**  
*Surveyor & Lieutenant Colonel* (1732‑1775)  
- First officer to deliver ultimatum to French forces at Fort Duquesne (1754).  
- Commanded Virginia forces after General Braddock’s death, demonstrating battlefield adaptability.  
- Conducted frontier surveying, measuring vast tracts of land, and creating accurate maps for colonial expansion.

**Plantation Manager & Estate Administrator – Mount Vernon**  
*Owner & Manager* (1755‑1775)  
- Sustained a large plantation, cultivating agricultural expertise and large‑scale estate operations.  
- Built social capital among Virginia’s elite, establishing a network that later supported revolutionary 
activities.

---

## Education  

- **Apprenticeship, Surveying & Cartography** – *Lord Fairfax* (early 1700s)  
  • Learned frontier surveying, land measurement, and map‑making through practical field training.  
- **Self‑Directed Studies** – *Military Tactics & Colonial Governance*  
  • Acquired deep knowledge of military strategy, logistics, and governance via active participation in colonial 
politics.

---

## Key Accomplishments  

| Year | Achievement | Impact |
|------|-------------|--------|
| 1783 | Treaty of Paris | Secured American independence from Great Britain. |
| 1787 | Constitutional Ratification | Established the U.S. Constitution and federal governance. |
| 1790 | First Bank of the U.S. | Created a stable national currency and credit system. |
| 1793 | Proclamation of Neutralit

In [17]:
with open("GEORGE_WASHINGTON_RESUME.md", "w", encoding="utf-8") as f:
    f.write(resume)